# Iceberg Tracking
This notebook documents the components and the pipeline of the iceberg tracking system, including data preparation, detection, tracking and visualization steps.

## Components
The tracking-pipeline consists of 4 components:

- `ImagePreprocessor`: For preprocessing the input images.
- `IcebergDetector`: For detecting icebergs in the images.
- `IcebergTracker`: For tracking detected icebergs across image frames.
- `Visualizer`: For visualizing the results.

#### ImagePreprocessor
The `ImagePreprocessor` is responsible for applying optional images processing operations to to improve the visibility of icebergs for the detection model. For each dataset it has only to be executed once. This instance is configured to: 

- `brighten`: Whether to apply brightness adjustment for night hours
- `mask`: Whether to apply masking to specific areas that are not relevant (e.g., the sky, the glacier) - a masking image needs to be provided in the image directory in advance
- `tile`: Whether to split images into tiles for improved detection model performance - the tile coordinates need to be provided in preprocessing.py
- `night_start`: Hour (0-23) marking start of night for brightness enhancement
- `night_end`: Hour (0-23) marking end of night for brightness enhancement

The input images need to be stored in `data/{dataset_name}/images/raw/`

#### Iceberg Detector
The `IcebergDetector`-class adapts a pre-trained Faster R-CNN model with ResNet50 backbone and Feature Pyramid Network (FPN) using PyTorch. 
It consists of components:
- `train()`: Train the model using k-fold cross-validation with early stopping, timing metrics and best model selection based on validation loss. It can be configured as follows:
    - `k_folds`: Number of folds for cross-validation, dividing the dataset into k parts for rotating validation
    - `num_epochs`: Maximum number of training epochs per fold
    - `patience`: Number of consecutive epochs with no improvement before triggering early stopping
    - `batch_size`: Number of samples per batch for training and validation
    - `learning_rate`: Learning rate for the optimizer
- `predict()`: Loads the trained model, processes images in the dataset directory, generates bounding box predictions for each detected icebergs, postprocess the results and saves them to a detection file. It can be configured as follows:
    - `confidence_threshold`: Minimum confidence score for detections to be included in the output
 
#### IcebergTracker
The `IcebergTracker`-class is responsible for associating detected icebergs across consecutive frames to maintain a consistent track for each individual iceberg based on the detection data. 

The tracking algorithm is divided into 3 phases:
1. Preselect candidates based on size, intersection and distance threshold
2. Match icebergs among preselected candidates via cost matrix optimization of distance, size and intersection
3. Apply post processing filters to avoid tracking of wrongly detected icebergs and nested icebergs

The `track()`-method can be configured as follows:
- `cost_threshold`: Maximum score threshold of the cost matrix for icebergs to be matched
- `distance_top_k`: Consider only the top X% closest new icebergs
- `size_threshold`: Minimum size similarity for a valid match
- `iou_threshold`: Minimum IoU to consider a valid match
- `distance_weight`: Weight for distance in cost calculation
- `size_weight`: Weight for size similarity in cost calculation
- `iou_weight`: Weight for IoU in cost calculation
- `max_inactive_frames`: Maximum number of frames an iceberg can be inactive before being discarded - enables consisten track of an iceberg even when it's not detected in every frame 
- `perspective_k`: Factor for correcting perspective distortion in distance calculations, considering that distances in the background appear smaller than they are
- `min_detection_count`: Minimum number of detections required to keep an iceberg track - avoids tracking of wrongly detected icebergs (e.g., shadow in the water)
- `nested_threshold`: Threshold for filtering nested icebergs (0.8 means 80% overlap) to avoid the tracking of icebergs within an iceberg

#### Visualizer
You can visualize each step with the `Visualizer`-class.
It can display raw images, preprocessed images, detection results or tracking results and the instance is configured to:

- `stage`: Processing stage to visualize - Options: 'preprocessing', 'detection' or 'tracking'
- `image_dir`: Directory containing the images - Options: 'raw' or 'preprocessed'
- `segment`: If True, visualize segmentation, else bounding boxes
- `start_index`: Starting index for image selection
- `length`: Number of images to process

The visualization can be displayed via two different methods: 

- `pylot()`: Display all images individually - useful for detailed inspection of individual frames.
- `render_videos()`: Compile the visualization results into a video with configurion options of frames per second and resolution 



## Pipeline

#### Step 1: Import Required Modules
Imports all necessary custom classes for the iceberg tracking pipeline. 

In [3]:
from preprocessing import ImagePreprocessor 
from detection import IcebergDetector
from tracking import IcebergTracker
from utils.visualize import Visualizer

#### Step 2: Define Dataset Parameters
Specify the dataset name and image format to be used throughout the pipeline.

- `dataset`: Name of the dataset stored in `data/` containing the iceberg images
- `image_format`: The file extension of the images (e.g., JPG, PNG).

In [5]:
dataset = "hill_2min_2023-08"
image_format = "JPG"

#### Step 3: Preprocess Images

In [ ]:
# Creates an ImagePreprocessor instance with the specified dataset and preprocessing options
preprocessor = ImagePreprocessor(
    dataset=dataset,
    image_format=image_format,
    brighten=True,
    mask=True,
    tile=True,
    night_start=0,  # 0 AM
    night_end=6  # 6 AM
)

# Run the preprocessing pipeline
preprocessor.process_images()

In [ ]:
# Display image before preprocessing 
visualizer = Visualizer(dataset, image_format=image_format, stage="raw", image_dir="processed", start_index=0, length=1)
visualizer.pyplot()

In [ ]:
# Display image after preprocessing 
visualizer = Visualizer(dataset, image_format=image_format, stage="preprocessing", image_dir="processed", start_index=0, length=5)
visualizer.pyplot()

#### Step 4: Prepare Training Data

The input data with iceberg detections used for training needs to be stored in `data/{dataset_name}/annotations/gt.txt`. This file follows the MOT format where each line consists of the following 10 values based on the preprocessed images:
```
<frame>, <id>, <bb_left>, <bb_top>, <bb_width>, <bb_height>, <conf>, <x>, <y>, <z>
```

Here is an example:
```
_MG_17310_B,1,161.15,1087.59,247.32,90.46,0.9999,-1,-1,-1
_MG_17310_B,2,257.76,1200.62,125.16,50.46,0.9999,-1,-1,-1
```

#### Step 5: Detect Icebergs

In [6]:
# Create detector instance
detector = IcebergDetector(dataset, image_format=image_format)

In [ ]:
# Train model based on gt.txt data
detector.train(k_folds=2, num_epochs=1, patience=3)  # Recommended: k_folds=5 and num_epochs=10, lower values only for demonstration purposes 

In [ ]:
# Detects icebergs and and saves bounding boxes to det.txt file
detector.predict(confidence_threshold=0.0)

In [ ]:
# Display detected icebergs
visualizer = Visualizer(dataset, image_format=image_format, stage="detection", start_index=0, length=10)
visualizer.render_video()

#### Step 6: Track Icebergs

In [ ]:
# Create tracking instance
tracker = IcebergTracker(dataset, image_format=image_format)
# Match and track icebergs
tracker.track(
    cost_threshold=0.3,
    distance_top_k=0.1,
    size_threshold=0.3,
    iou_threshold=0.00,
    distance_weight=1.0,
    size_weight=1.0,
    iou_weight=1.0,
    max_inactive_frames=2,
    perspective_k=3,
    min_detection_count=10,
    nested_threshold=0.8,
)

In [ ]:
# Display tracked icebergs with bounding boxes
visualizer = Visualizer(dataset, image_format=image_format, stage="tracking", start_index=0, length=10)
visualizer.render_video()
# Display tracked segmented icebergs
visualizer = Visualizer(dataset, image_format=image_format, stage="tracking", segment=True, start_index=0, length=10)
visualizer.render_video()